In [145]:
import numpy as np
from implementations import *
from cross_validation import *
from data_preprocessing import *
from proj1_helpers import *
import math

print('loading data'+"\n")
DATA_TEST_PATH = '../data/train.csv'
y,tX,ids = load_csv_data(DATA_TEST_PATH)
print('data loaded')

jet_set = jet(tX)
inds = create_inds(jet_set, False)

data_sets = jet_split(tX,inds)
y_sets = split_y(y,inds)

loading data

data loaded


In [146]:
def replace_aberrant_values(tX):
    '''Replaces the aberrant value (-999) for a given feature 
    and  replaces it by the mean observed value of that feature.'''
    tX_repl_feat = np.copy(tX)
    means = []
    
    #compute the mean of each feature (column) without taking -999 values into account
    for j in range(tX_repl_feat.shape[1]):
        m = tX_repl_feat[:,j][tX_repl_feat[:,j] != -999].mean()
        means.append(m)
    
    #change all -999 values of a column by the mean computed previously
    for i in range(len(means)):
        mask = tX_repl_feat[:, i] == -999
        tX_repl_feat[:, i][mask] = means[i]
    
    return tX_repl_feat

In [147]:
def standardize(x):

    centered_data = x - np.mean(x, axis=0)
    std_data = centered_data / np.std(centered_data, axis=0)
    
    return std_data

In [148]:
def cross_validation(y, tX, lambda_, degree, ratio):
    
    for i in range(int(1/(1-ratio))):

        xtrain, ytrain, xtest, ytest = split_data(tX, y, ratio)
        
        weights_ = []
        trainlosses = []
        testlosses = []

        data_set=build_poly(xtrain,degree)
        data_set_test=build_poly(xtest,degree)

        w, loss = ridge_regression(ytrain,data_set,lambda_)

        weights_.append(w)
        trainlosses.append(loss)
        testlosses.append(compute_loss(ytest,data_set_test,w))
            
    #print("test error =",np.mean(testlosses))
    #print("train error =", np.mean(trainlosses))

    return np.mean(testlosses), np.mean(trainlosses)

In [149]:
for i in range(len(data_sets)):
    data_sets[i] = replace_aberrant_values(data_sets[i])
    data_sets[i] = standardize(data_sets[i])
    print('hello')

hello
hello
hello


In [154]:
np.where(data_sets[0] == -999)[0]

array([], dtype=int64)

In [ ]:
matrix = [[1, 1, 1, 1, 1]]
weights_ = []
losses = []
lambda_= np.logspace(1, -8, num = 10)
degree = np.linspace(0,10, num = 10, dtype = int)

for lam in lambda_:
    for deg in degree:       
        for data_set, y_set in zip(data_sets, y_sets):
            test_error, train_error = cross_validation(y_set, data_set, lam, deg, 0.8)
            losses.append(test_error)
            #losses.append(train_error)
        matrix = np.append(matrix, [[lam, deg, losses[0], losses[1], losses[2]]], axis = 0)
